In [ ]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
from datetime import datetime
from math import ceil
import seaborn as sns
from dateutil.relativedelta import relativedelta
from sklearn.metrics import mean_squared_error
from datetime import timedelta
import matplotlib.pyplot as plt

from math import sqrt
import holidays
import catboost as ctb

In [ ]:
df = pd.read_csv('/kaggle/input/sales_transformed_with_coords_metro.csv')

## Feature Engineering

In [ ]:
df['date'] = df.apply(lambda x: datetime(int(x['year']), int(x['month']), 1), axis=1)

In [ ]:
# find empty uninformative brand with no sales for whole period
df_brand_sum = df.groupby(['brand'])['sales'].sum().reset_index()
useless_brands = df_brand_sum[df_brand_sum['sales'] <= 0]['brand'].tolist()

In [ ]:
df = df.drop(df[df['brand'] == useless_brands[0]].index)

In [ ]:
# drop Nan rows
df = df.dropna(subset=['sales', 'district'])

#### Lag features for store sales

In [ ]:
month_revenues = df.groupby(['address', 'date']).agg({'sales': ['sum', 'mean', 'std']}).reset_index()
month_revenues.columns = ['address', 'date', 'sales_sum', 'sales_mean', 'sales_std']

In [ ]:
month_revenues['lag_1_store_sales_sum'] =  month_revenues.groupby('address')['sales_sum'].shift(1)
month_revenues['lag_1_store_sales_mean'] =  month_revenues.groupby('address')['sales_mean'].shift(1)
month_revenues['lag_1_store_sales_std'] =  month_revenues.groupby('address')['sales_std'].shift(1)

month_revenues['lag_2_store_sales_sum'] =  month_revenues.groupby('address')['sales_sum'].shift(2)
month_revenues['lag_2_store_sales_mean'] =  month_revenues.groupby('address')['sales_mean'].shift(2)
month_revenues['lag_2_store_sales_std'] =  month_revenues.groupby('address')['sales_std'].shift(2)

month_revenues['lag_3_store_sales_sum'] =  month_revenues.groupby('address')['sales_sum'].shift(3)
month_revenues['lag_3_store_sales_mean'] =  month_revenues.groupby('address')['sales_mean'].shift(3)
month_revenues['lag_3_store_sales_std'] =  month_revenues.groupby('address')['sales_std'].shift(3)

In [ ]:
left_df =  df.set_index(['address', 'date'])
right_df = month_revenues.set_index(['address', 'date'])
merged_df = pd.merge(left_df, right_df, left_index=True, right_index=True, how='left', sort=False).reset_index()

In [ ]:
merged_df[merged_df['address'] == 'Россия, г Москва, ш Ярославское, д 63'].tail(27)

#### Lag features for brand sales in each store

In [ ]:
df_grouped_by_brand = df.groupby(['address', 'brand', 'date']).agg({'sales':'sum'}).reset_index()

In [ ]:
df_grouped_by_brand['lag_1_brand_sales'] = df_grouped_by_brand.groupby(['address', 'brand'])['sales'].shift(1)
df_grouped_by_brand['lag_2_brand_sales'] = df_grouped_by_brand.groupby(['address', 'brand'])['sales'].shift(2)
df_grouped_by_brand['lag_3_brand_sales'] = df_grouped_by_brand.groupby(['address', 'brand'])['sales'].shift(3)

In [ ]:
df_grouped_by_brand[df_grouped_by_brand['brand'] == 'Царская Чарка'].tail(21)

In [ ]:
left_df =  merged_df.set_index(['address', 'brand', 'date'])
right_df = df_grouped_by_brand.drop('sales', axis=1).set_index(['address', 'brand', 'date'])
df = pd.merge(left_df, right_df, left_index=True, right_index=True, how='left', sort=False).reset_index()

In [ ]:
df.loc[(df['address'] == 'Россия, г Москва, ш Ярославское, д 69') & (df['brand'] == 'Царская Чарка')].tail(21)

#### Days, Holidays count

In [ ]:
days = pd.Series([0, 31,28,31,30,31,30,31,31,30,31,30,31])
df['days'] = df['month'].map(days).astype(np.int64)
df['days'] = df['days'].astype(np.int64)

In [ ]:
ru_holidays = holidays.Russia()

def get_holidays_count(month_start):
    month_start = pd.to_datetime(month_start)
    month_end = month_start + relativedelta(months=1)
    return len(ru_holidays[month_start:month_end])

In [ ]:
df['holidays_cnt'] = df['date'].apply(get_holidays_count).astype(np.int64)

In [ ]:
df = df.drop(['location', 'Unnamed: 0'], axis=1)

## Train/Validation/Test split

I will use last month for test set

In [ ]:
train_set = df[df['date'] < '2019-12-01'].reset_index()
test_set = df[df['date'] >= '2019-12-01'].reset_index()

In [ ]:
# sanity check
len(test_set) + len(train_set) == len(df)

In [ ]:
def train_val_split(df, start_date):
    start_date = pd.to_datetime(start_date)
    in_three_months = (start_date + relativedelta(months=3)).replace(day=1)
    in_four_months = (start_date + relativedelta(months=4)).replace(day=1)
    return train_set.loc[
        (train_set['date'] >= start_date) & (train_set['date'] < in_three_months)
    ].index,\
    train_set.loc[
        (train_set['date'] >= in_three_months) & (train_set['date'] < in_four_months)
    ].index

In [ ]:
def get_cv_iterator(dataset):
    for year in (2018, 2019):
        for month in range(1, 13):
            d = '{}-{}-01'.format(year, month)
            if d == '2019-9-01':
                break
            yield train_val_split(dataset, d)

In [ ]:
cv_iterator = get_cv_iterator(train_set)

In [ ]:
X = train_set.drop(['sales', 'date', 'address'], axis=1)
y = train_set['sales']

X_test = test_set.drop(['sales', 'date', 'address'], axis=1)
y_test = test_set['sales']

In [ ]:
# sanity check
len(X_test) + len(X) == len(df), len(y) + len(y_test) == len(df)

## Baseline models

In [ ]:
def get_rmse(y_actual, y_predicted):
    return sqrt(mean_squared_error(y_actual, y_predicted))

In [ ]:
def get_rmse_per_brand(y_actual, y_predicted):
    idx = y_actual.index
    stats = []
    brands = X_test.loc[X_test.index.isin(idx)]['brand'].unique().tolist()
    y_pred_df = pd.DataFrame({'y_pred': y_predicted})
    y_actual_pred_df = pd.concat([y_actual, y_pred_df], axis=1)
    for brand in brands:
        brand_idx = X_test.loc[(X_test.index.isin(idx)) & (X_test['brand'] == brand)].index
        smse = sqrt(mean_squared_error(y_actual.loc[brand_idx], y_pred_df.loc[brand_idx]))
        stats.append((brand, smse))
    return list(sorted(stats))

1. Constant

In [ ]:
CONS = 1
rmses = []
for _, val_idx in cv_iterator:
    y_actual = y.loc[val_idx]
    y_predicted = [CONS] * len(y_actual)
    rmses.append(get_rmse(y_actual, y_predicted))
np.mean(rmses)

2. Prediction by last period

In [ ]:
def get_prev_sales(row):
        prev_date = pd.to_datetime(row['date']) - relativedelta(months=1)
        prev_date = prev_date.replace(day=1)
        last_y = train_set[
            (train_set['month'] == prev_date) & 
            (train_set['brand'] == row['brand'])
        ]['sales']
        if last_y.any():
            last_y = last_y.iloc[0]['sales']
        else:
            last_y = 0
        return last_y

In [ ]:
rmses = []
for _, val_idx in cv_iterator:
    y_actual = y.loc[val_idx]
    y_predicted = train_set.loc[val_idx].apply(get_prev_sales, axis=1)
    rmses.append(get_rmse(y_actual, y_predicted))
np.mean(rmses)

3. Mean for last 5 periods

In [ ]:
def get_prev_5_sales(row):
        y_previous = []
        for i in range(1, 6):
            prev_date = pd.to_datetime(row['date']) - relativedelta(months=i)
            prev_date = prev_date.replace(day=1)
            prev_y = train_set[
                (train_set['month'] == prev_date) & 
                (train_set['brand'] == row['brand'])
            ]['sales']
            if prev_y.any():
                prev_y = prev_y.iloc[0]['sales']
            else:
                prev_y = 0
            y_previous.append(prev_y)
        return np.mean(y_previous or [0])

In [ ]:
rmses = []
for _, val_idx in cv_iterator:
    y_actual = y.loc[val_idx]
    y_predicted = train_set.loc[val_idx].apply(get_prev_5_sales, axis=1)
    rmses.append(get_rmse(y_actual, y_predicted))
np.mean(rmses)

4. Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

rmses = []
for train_idx, val_idx in cv_iterator:
    lin_reg = LinearRegression()
    lin_reg.fit(X.loc[train_idx], y.loc[train_idx])
    y_val_predicted = lin_reg.predict(train_set.loc[val_idx])
    rmse = get_rmse(y.loc[val_idx], y_val_predicted)
    rmses.append(rmse)

In [ ]:
np.mean(rmses)

## Catboost

In [ ]:
len(X.columns)
categorical_features_indices = np.where((X.dtypes != np.float) & (X.dtypes != np.int))[0]

In [ ]:
X.dtypes

In [ ]:
categorical_features_indices

In [ ]:
params = {
    "loss_function": "RMSE",
    "eval_metric": "RMSE",
    "iterations": 1200,
    "random_seed": 42,
    "od_wait": 50,
    "od_type": "Iter",
    "thread_count": 10
}

grid = {
    'learning_rate': [0.03, 0.06, 0.08],
    'depth': [10, 12, 15],
}

ctb_data = ctb.Pool(X, y, cat_features=categorical_features_indices)

model = ctb.CatBoostRegressor(**params)

In [ ]:
grid_search_result = model.grid_search(grid, ctb_data, plot=True, cv=cv_iterator)

In [ ]:
# best_params = grid_search_result['params']
best_params = dict(learning_rate=0.06, depth=12)
params.update(best_params)

In [ ]:
scores = ctb.cv(ctb_data, params, plot="True", folds=cv_iterator)

In [ ]:
model = ctb.CatBoostRegressor(**params)
model.fit(ctb_data)

In [ ]:
y_pred = model.predict(X_test)
get_rmse(y_test, y_pred)

#### Plot RMSE per each brand

In [ ]:
per_brand_stats = sorted(get_rmse_per_brand(y_test, y_pred), key=lambda x: -x[1])
per_brand_stats

In [ ]:
plt.figure(figsize=(40,8))
plt.xticks(rotation='vertical')
plt.bar([x[0] for x in per_brand_stats[90:]], [x[1] for x in per_brand_stats[90:]])

#### Plot Feature importances

In [ ]:
feature_importances = sorted(zip(X.columns, model.get_feature_importance(verbose=True)), key=lambda k: -k[1])
feature_importances

In [ ]:
plt.figure(figsize=(40,8))
plt.xticks(rotation='vertical')
plt.bar([x[0] for x in feature_importances], [x[1] for x in feature_importances])

In [ ]:
model.save_model('catboost_model_with_pool', pool=ctb_data)

## Light GMB

In [ ]:
import lightgbm as lgb

In [ ]:
for col in ['brand', 'store_format', 'district']:
    X[col] = X[col].astype('category')
    X_test[col] = X_test[col].astype('category')

In [ ]:
n_rounds = 100000

parameters = {
    #default
    "learning_rate": 0.01,
    "num_threads": 10,
    "metric": "rmse",
    "seed": 42,
    
    #regularization
    "colsample_bytree": 0.8,
    "subsample": 0.8,
    "subsample_freq": 1,
    "min_data_in_leaf": 15,
    
    #categorical features
    'cat_smooth': 10,
    'min_data_per_group': 50,
}
lgb_train = lgb.Dataset(X, label=y, free_raw_data=False, categorical_feature=['brand', 'store_format', 'district'])
result = lgb.cv(
    parameters, 
    lgb_train, 
    n_rounds, 
    folds=cv_iterator, 
    early_stopping_rounds=50, 
    verbose_eval=100, 
    eval_train_metric=True, 
)

## XGBoost

In [ ]:
dummy_district = pd.get_dummies(df['district'])
dummy_brand = pd.get_dummies(df['brand'])
dummy_store_format = pd.get_dummies(df['store_format'])

In [ ]:
df = pd.concat([df, dummy_district, dummy_brand, dummy_store_format], axis=1)

In [ ]:
df.head()

In [ ]:
train_set = df[df['date'] < '2019-12-01']

In [ ]:
train_set.head()

In [ ]:
cv_iterator = list(get_cv_iterator(train_set))

In [ ]:
X = train_set.drop(['sales', 'date', 'address', 'district', 'store_format', 'brand'], axis=1)
y = train_set['sales']

In [ ]:
# clear RAM
del month_revenues, left_df, right_df, merged_df, df_grouped_by_brand
del dummy_district, dummy_brand, dummy_store_format
del df, train_set

import gc
gc.collect()

In [ ]:
import xgboost as xgb

In [ ]:
parameters = {
    #default
    "objective": "reg:squarederror",
    "eta": 0.01,
    "verbosity": 0,
    "nthread": 10,
    "random_seed": 1,
    "eval_metric": "rmse",
    "max_depth": 10,
    
    "subsample": 1,
    "colsample_bytree": 1,
    
    "tree_method": "hist",
    "grow_policy": "lossguide"
}


model = xgb.XGBRegressor(**parameters)

In [ ]:
cv_rmses = []
for train_idx, val_idx in cv_iterator:
    model = xgb.XGBRegressor(**parameters)
    model.fit(X.loc[train_idx], y[train_idx], eval_metric='rmse')
    y_pred = model.predict(X.loc[val_idx])
    rmse = get_rmse(y[val_idx], y_pred)
    print('val RMSE: {}'.format(rmse))
    cv_rmses.append(rmse)

In [ ]:
import numpy as np
np.mean(cv_rmses), np.std(cv_rmses)